In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
207,AFG,Asia,Afghanistan,2020-08-04,36747.0,37.0,1288.0,4.0,943.965,0.950,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
356,ALB,Europe,Albania,2020-08-04,5620.0,101.0,172.0,6.0,1952.881,35.096,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
569,DZA,Africa,Algeria,2020-08-04,31972.0,507.0,1239.0,8.0,729.105,11.562,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-03,France,188209.0,0.0,30291.0,0.0,0.0,0.0
0,2020-08-03,China,88119.0,0.0,4677.0,0.0,0.0,0.0
0,2020-08-03,Italy,248468.0,0.0,35181.0,0.0,0.0,0.0
0,2020-08-03,Spain,288730.0,0.0,28455.0,0.0,0.0,0.0
0,2020-08-03,United States,4754600.0,0.0,156079.0,0.0,0.0,0.0
0,2020-08-03,World,18477737.0,0.0,696880.0,0.0,0.0,0.0
0,2020-08-03,United Kingdom,305680.0,0.0,46321.0,0.0,0.0,0.0
0,2020-08-03,Germany,210731.0,0.0,9155.0,0.0,0.0,0.0
0,2020-08-03,Iran,312999.0,0.0,17404.0,0.0,0.0,0.0
0,2020-08-03,Turkey,233784.0,0.0,5745.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-04,France,188209.0,191295.0,30291.0,30294.0,-3086.0,-3.0
0,2020-08-04,China,88119.0,88099.0,4677.0,4672.0,20.0,5.0
0,2020-08-04,Italy,248468.0,248229.0,35181.0,35166.0,239.0,15.0
0,2020-08-03,Spain,288730.0,297054.0,28455.0,28472.0,-8324.0,-17.0
0,2020-08-04,United States,4754600.0,4713562.0,156079.0,155403.0,41038.0,676.0
0,2020-08-04,World,18477737.0,18263542.0,696880.0,693726.0,214195.0,3154.0
0,2020-08-04,United Kingdom,305680.0,305623.0,46321.0,46210.0,57.0,111.0
0,2020-08-04,Germany,210731.0,211281.0,9155.0,9156.0,-550.0,-1.0
0,2020-08-04,Iran,312999.0,312035.0,17404.0,17405.0,964.0,-1.0
0,2020-08-04,Turkey,233784.0,233851.0,5745.0,5747.0,-67.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")